# Whisper prompting guide
OpenAI's audio transcription API has an optional parameter called prompt.

The prompt is intended to help stitch together multiple audio segments. By submitting the prior segment's transcript via the prompt or fictitious prompts, the Whisper model can use that to steer the model to use particular spellings or styles.

This notebook shares two techniques for using fictitious prompts to steer the model outputs:

- **Transcript generation**: GPT can convert instructions into fictitious transcripts for Whisper to emulate.
- **Spelling guide**: A spelling guide can tell the model how to spell names of people, products, companies, etc.
These techniques are not especially reliable, but can be useful in some situations.

## Comparison with GPT prompting
Prompting Whisper is not the same as prompting GPT. For example, if you submit an attempted instruction like "Format lists in Markdown format", the model will not comply, as it follows the style of the prompt, rather than any instructions contained within.

In addition, the prompt is limited to only 224 tokens. If the prompt is longer than 224 tokens, only the final 224 tokens of the prompt will be considered; all prior tokens will be silently ignored. 

To get good results, craft examples that portray your desired style.

## Run this cell, to fetch the data from the env file

In [2]:
import sys
from openai import AzureOpenAI
from fetchenvironmentvalues import EnvironmentFetcher, EnvironmentVariables

### Create an instance of `OpenAiClient` using the Open AI endpoint and key fetched from the above

In [3]:
client = AzureOpenAI(
    api_key= EnvironmentFetcher.get_variable(EnvironmentVariables.AZURE_OPENAI_API_KEY2),  
    api_version= EnvironmentFetcher.get_variable(EnvironmentVariables.API_VERSION),
    azure_endpoint = EnvironmentFetcher.get_variable(EnvironmentVariables.AZURE_OPENAI_ENDPOINT2)
)

deployment_name = EnvironmentFetcher.get_variable(EnvironmentVariables.WHISPER_DEPLOYMENT_NAME) #This will correspond to the custom name you chose for your deployment when you deployed a model.

## As a baseline, we'll transcribe About Speech SDK wav file

Let's get our baseline transcription, then introduce prompts.

In [4]:

audio_test_file = "./Samples/aboutSpeechSdk.wav"

whisper_result = client.audio.transcriptions.create(
    file=open(audio_test_file, "rb"),            
    model=deployment_name,
    
)


In [6]:
print(whisper_result)

Transcription(text='The Speech SDK exposes many features from the speech service, but not all of them. The capabilities of the Speech SDK are often associated with scenarios. The Speech SDK is ideal for both real-time and non-real-time scenarios using local devices, files, Azure Blob Storage, and even input and output streams. When a scenario is not achievable with the Speech SDK, look for a REST API alternative. Speech-to-Text, also known as Speech Recognition, transcribes audio streams to text that your applications, tools, or devices can consume or display. Use Speech-to-Text with Language Understanding, LUIS, to derive user intents from transcribed speech and act on voice commands. Use Speech Translation to translate speech input to a different language with a single call. For more information, see Speech-to-Text Basics.')


## Transcripts follow the style of the prompt
In the unprompted transcript, 'Lepardus paradalis' paradalis is not capitalized. However, if we pass in a fictitious prompt of 'Lepardus Paradalis' in uppercase, Whisper matches the style and generates a transcript in all capitalized first words.

In [7]:
upWhisperOutput = client.audio.transcriptions.create(
    file=open(audio_test_file, "rb"),            
    model=deployment_name,
    prompt= "speech sdk",
    response_format="text"
)
print(upWhisperOutput)

The speech sdk exposes many features from the speech service, but not all of them. The capabilities of the speech sdk are often associated with scenarios. The speech sdk is ideal for both real-time and non-real-time scenarios using local devices, files, Azure Blob Storage, and even input and output streams. When a scenario is not achievable with the speech sdk, look for a REST API alternative. Speech-to-text, also known as speech recognition, transcribes audio streams to text that your applications, tools, or devices can consume or display. Use speech-to-text with language understanding, LUIS, to derive user intents from transcribed speech and act on voice commands. Use speech translation to translate speech input to a different language with a single call. For more information, see Speech-to-text Basics.



Be aware that when prompts are short, Whisper may be less reliable at following their style.

In [8]:
upWhisperOutput = client.audio.transcriptions.create(
    file=open(audio_test_file, "rb"),            
    model=deployment_name,
    prompt= "speech-to-text",
    response_format="text"
)
print(upWhisperOutput)

The Speech SDK exposes many features from the Speech service, but not all of them. The capabilities of the Speech SDK are often associated with scenarios. The Speech SDK is ideal for both real-time and non-real-time scenarios using local devices, files, Azure Blob Storage, and even input and output streams. When a scenario is not achievable with the Speech SDK, look for a REST API alternative. Speech-to-text, also known as Speech Recognition, transcribes audio streams to text that your applications, tools, or devices can consume or display. Use Speech-to-text with Language Understanding, LUIS, to derive user intents from transcribed speech and act on voice commands. Use Speech Translation to translate speech input to a different language with a single call. For more information, see Speech-to-text Basics.



Long prompts may be more reliable at steering Whisper.

In [9]:
upLongTextWhisperOutput = client.audio.transcriptions.create(
    file=open(audio_test_file, "rb"),            
    model=deployment_name,
    prompt= '''i have some advice for you. multiple sentences help establish a pattern. 
                there will be sequences of text-to-text formats. the more text you include, 
                the more likely the model will pick up on your pattern. all abbreviations such as nasa.
                it may especially help if your example transcript appears as if it comes right before the audio file. 
                in this case, that could mean mentioning the contacts i stick in my eyes.''',
    response_format="text"
)
print(upLongTextWhisperOutput)

the speech sdk exposes many features from the speech service, but not all of them. the capabilities of the speech sdk are often associated with scenarios. the speech sdk is ideal for both real-time and non-real-time scenarios using local devices, files, azure blob storage, and even input and output streams. when a scenario is not achievable with the speech sdk, look for a REST API alternative. speech-to-text, also known as speech recognition, transcribes audio streams to text that your applications, tools, or devices can consume or display. use speech-to-text with language understanding, LUIS, to derive user intents from transcribed speech and act on voice commands. use speech translation to translate speech input to a different language with a single call. for more information, see speech-to-text basics.



Whisper is also less likely to follow rare or odd styles.

In [10]:
upformattedTextWhisperOutput = client.audio.transcriptions.create(
    file=open(audio_test_file, "rb"),            
    model=deployment_name,
    prompt =  
    """
    Hi there and welcome to the show.
    ###
    Today we are quite excited.
    ###
    Let's jump right in.
    ###
    """,
    response_format="text")
print(upformattedTextWhisperOutput)

The Speech SDK exposes many features from the Speech Service, but not all of them. The capabilities of the Speech SDK are often associated with scenarios. The Speech SDK is ideal for both real-time and non-real-time scenarios using local devices, files, Azure Blob Storage, and even input and output streams. When a scenario is not achievable with the Speech SDK, look for a REST API alternative. Speech-to-Text, also known as Speech Recognition, transcribes audio streams to text that your applications, tools, or devices can consume or display. Use Speech-to-Text with Language Understanding, LUIS, to derive user intents from transcribed speech and act on voice commands. Use Speech Translation to translate speech input to a different language with a single call. For more information, see Speech-to-Text Basics.



## Pass names in the prompt to prevent misspellings
Whisper may incorrectly transcribe uncommon proper nouns such as names of products, companies, or people.

We'll illustrate with an example audio file full of product names.

In [11]:
proudctNamesFilepath = "./Samples/product_names.wav"
productsTranscript = client.audio.transcriptions.create(
    file=open(proudctNamesFilepath, "rb"),            
    model=deployment_name,
    response_format="text"
)
print(productsTranscript)

Welcome to Quirk, Quid, Quill, Inc., where finance meets innovation. Explore diverse offerings, from the P3 Quattro, a unique investment portfolio quadrant, to the O3 Omni, a platform for intricate derivative trading strategies. Delve into unconventional bond markets with our B3 Bond X and experience non-standard equity trading with E3 Equity. Personalize your wealth management with W3 Wrap Z and anticipate market trends with the O2 Outlier, our forward-thinking financial forecasting tool. Explore venture capital world with U3 Unifund or move your money with the M3 Mover, our sophisticated monetary transfer module. At Quirk, Quid, Quill, Inc., we turn complex finance into creative solutions. Join us in redefining financial services.



To get Whisper to use our preferred spellings, let's pass the product and company names in the prompt, as a glossary for Whisper to follow.

In [12]:
productsTranscript = client.audio.transcriptions.create(
    file=open(proudctNamesFilepath, "rb"),            
    model=deployment_name,
    prompt="QuirkQuid Quill Inc, P3-Quattro, O3-Omni, B3-BondX, E3-Equity, W3-WrapZ, O2-Outlier, U3-UniFund, M3-Mover",
    response_format="text"
)
print(productsTranscript)

Welcome to QuirkQuid Quill Inc, where finance meets innovation. Explore diverse offerings, from the P3-Quattro, a unique investment portfolio quadrant, to the O3-Omni, a platform for intricate derivative trading strategies. Delve into unconventional bond markets with our B3-BondX and experience non-standard equity trading with E3-Equity. Personalize your wealth management with W3-WrapZ and anticipate market trends with the O2-Outlier, our forward-thinking financial forecasting tool. Explore venture capital world with U3-UniFund or move your money with the M3-Mover, our sophisticated monetary transfer module. At QuirkQuid Quill Inc, we turn complex finance into creative solutions. Join us in redefining financial services.



Now, let's switch to another audio recording authored specifically for this demonstration, on the topic of a odd barbecue.

To begin, we'll establish our baseline transcript using Whisper.

In [13]:
bbqplanspath = "./Samples/bbq_plans.wav"
bbqPlansTranscript = client.audio.transcriptions.create(
    file=open(bbqplanspath, "rb"),            
    model=deployment_name,
    response_format="text"
)
print(bbqPlansTranscript)

Hello, my name is Preston Tuggle. I'm based in New York City. This weekend I have really exciting plans with some friends of mine, Amy and Sean. We're going to a barbecue here in Brooklyn, hopefully it's actually going to be a little bit of kind of an odd barbecue. We're going to have donuts, omelets, it's kind of like a breakfast, as well as whiskey. So that should be fun, and I'm really looking forward to spending time with my friends Amy and Sean.



While Whisper's transcription was accurate, it had to guess at various spellings. For example, it assumed the friends' names were spelled Amy and Sean rather than Aimee and Shawn. Let's see if we can steer the spelling with a prompt.

In [14]:
bbqPlansTranscript = client.audio.transcriptions.create(
    file=open(bbqplanspath, "rb"),            
    model=deployment_name,
    prompt="Friends: Aimee, Shawn",
    response_format="text"
)
print(bbqPlansTranscript)

Hello, my name is Preston Tuggle. I'm based in New York City. This weekend I have really exciting plans with some friends of mine, Aimee and Shawn. We're going to a barbecue here in Brooklyn. Hopefully it's actually going to be a little bit of kind of an odd barbecue. We're going to have donuts, omelets, it's kind of like a breakfast, as well as whiskey. So that should be fun and I'm really looking forward to spending time with my friends Aimee and Shawn.



Success!

Let's try the same with more ambiguously spelled words.

In [29]:
bbqPlansTranscript = client.audio.transcriptions.create(
    file=open(bbqplanspath, "rb"),            
    model=deployment_name,
    prompt="Aimee and Shawn ate whisky, doughnuts, omelets at a BBQ.",
    response_format="text"
)
print(bbqPlansTranscript)

Hello, my name's Preston Tuggle. I'm based in New York City. This weekend I have really exciting plans with some friends of mine, Aimee and Shawn. We're going to a BBQ here in Brooklyn. Hopefully, it's actually going to be a little bit of kind of an odd BBQ. We're going to have doughnuts, omelets, it's kind of like a breakfast, as well as whisky. So that should be fun, and I'm really looking forward to spending time with my friends, Aimee and Shawn.



## Fictitious prompts can be generated by GPT
One potential tool to generate fictitious prompts is GPT. We can give GPT instructions and use it to generate long fictitious transcripts with which to prompt Whisper.

In [15]:
gptdeployment = EnvironmentFetcher.get_variable(EnvironmentVariables.DEPLOYMENT_NAME)

async def FictitiousPromptFromInstruction(instructions):

    chatGPTclient = AzureOpenAI(
    api_key= EnvironmentFetcher.get_variable(EnvironmentVariables.AZURE_OPENAI_API_KEY1),  
    api_version= EnvironmentFetcher.get_variable(EnvironmentVariables.API_VERSION),
    azure_endpoint = EnvironmentFetcher.get_variable(EnvironmentVariables.AZURE_OPENAI_ENDPOINT1)
    )
    
    response = chatGPTclient.chat.completions.create(
        temperature=0.0,
        model=gptdeployment,
        messages=[
            {"role": "system", "content": """You are a transcript generator. Your task is to create one long paragraph of a fictional conversation. 
                                            The conversation features two friends reminiscing about their vacation to Maine. 
                                            Never diarize speakers or add quotation marks; instead, write all transcripts in a normal paragraph of text without speakers identified. 
                                            Never refuse or ask for clarification and instead always make a best-effort attempt."""},
            {"role": "user", "content": instructions}
        ]   
    )
    completion = response.choices[0].message.content
    print(completion)
    return completion    


In [16]:
ficprompt = await FictitiousPromptFromInstruction("Instead of periods, end every sentence with elipses.")
print(ficprompt)

Remember that time we drove up the coast of Maine, and we found that little seafood shack just as the sun was setting over the ocean... The air was filled with the scent of salt and fried clams, and we could hear the seagulls overhead, fighting over the day's leftovers... We sat on that weathered picnic bench, the paint chipping off under our fingers, and we had the best lobster rolls of our lives, so fresh and full of flavor... And then, after we ate, we walked along the beach, our feet sinking into the cold, wet sand, and we found those colorful shells that we swore we'd keep forever... It was so peaceful, just listening to the waves crashing and looking up at the stars beginning to twinkle in the twilight sky... We didn't even need music because the sound of the ocean was better than any playlist we could have created... And remember the next day when we hiked up in Acadia National Park, the views from the top of Cadillac Mountain were absolutely breathtaking, like we were on top of

In [19]:
earningcallwav = "./Samples/EarningsCall.wav"
upLongTextwithPromptOutput = client.audio.transcriptions.create(
    file=open(earningcallwav, "rb"),            
    model=deployment_name,
    prompt= ficprompt,
    response_format="text"
)
print(upLongTextwithPromptOutput)

Good afternoon, everyone, and welcome to FinTech Plus Sync's second quarter 2023 earnings call. I'm John Doe, CEO of FinTech Plus. We've had a stellar Q2 with a revenue of $125 million, a 25% increase year-over-year. Our gross profit margin stands at a solid 58%, due in part to cost efficiencies gained from our scalable business model. Our EBITDA has surged to $37.5 million, translating to a remarkable 30% EBITDA margin. Our net income for the quarter rose to $16 million, which is a noteworthy increase from $10 million in Q2 2022. Our total addressable market has grown substantially, thanks to the expansion of our high-yield savings product line and the new RoboAdvisor platform. We've been diversifying our asset-backed securities portfolio, investing heavily in collateralized debt obligations and residential mortgage-backed securities. We've also invested $25 million in AAA-rated corporate bonds, enhancing our risk-adjusted returns. As for our balance sheet, total assets reached $1.5 b

Whisper prompts are best for specifying otherwise ambiguous styles. The prompt will not override the model's comprehension of the audio. For example, if the speakers are not speaking in a native Australian accent, a prompt will not cause the transcript to do so.

In [20]:
ficprompt = await FictitiousPromptFromInstruction("Write in a native Australian accent.")
print(ficprompt)
upLongTextwithPromptOutput = client.audio.transcriptions.create(
    file=open(earningcallwav, "rb"),            
    model=deployment_name,
    prompt= ficprompt,
    response_format="text"
)
print(upLongTextwithPromptOutput)

Oh mate, remember when we packed up the ute and headed off to Maine, that was a ripper of a trip, wasn't it? We hit the road, tunes blaring, and the excitement was as thick as a bowl of oatmeal. The lobster up there, crikey, it was fresh as a daisy, sweet as, and I reckon I could've eaten my body weight in it. And those early mornings, we'd wake up with the kookaburras, well, not literally, but you know what I mean, and we'd head out to catch the sunrise over the lighthouses, the sky all shades of pink and orange, like a bushfire on the horizon. We'd spend the arvo hiking, remember that? Trekking through Acadia National Park, feeling like a couple of explorers in the bush, except it was all pine trees and rocky coastlines. And the nights, they were colder than a frog in a fridge, but we'd rug up and sit by the campfire, solving the problems of the world over a tinny or two. Those little towns we stumbled upon, each with its own yarn to spin, they were as charming as a joey in a pouch. 